# Import

In [ ]:
#IMPORTATION DES LIBRAIRIES
from keras.layers import Conv2D,UpSampling2D
import keras
from keras import layers
import cv2
import zipfile
import math
from keras.layers import MaxPooling2D,Dropout
from keras.layers import InputLayer,Input,RepeatVector,Reshape,concatenate

from keras.models import Sequential,Model
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from skimage.color import rgb2lab,lab2rgb
from keras.datasets import cifar100
from skimage.transform import resize
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from tqdm import tqdm
from google.colab import drive
import os
from pathlib import Path
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import random
import pandas as pd


In [ ]:
#CONNECTION AU DRIVE 
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive")

In [ ]:
# zip = zipfile.ZipFile('some_images.zip')
# zip.extractall()

# Model

In [ ]:
IMG_SIZE = 256
N_EPOCHS = 30
BATCH_SIZE = 32
latent_dim = 256

In [ ]:
#DATA GENERATOR
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True)

train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/melange/tr',
                                              target_size = (IMG_SIZE, IMG_SIZE),
                                              batch_size = BATCH_SIZE)

valid_datagen = ImageDataGenerator(rescale = 1./255)

valid_set = valid_datagen.flow_from_directory('/content/drive/MyDrive/melange/vl',
                                              target_size = (IMG_SIZE, IMG_SIZE),
                                              batch_size = BATCH_SIZE)



def gen_ab_images(train_set):
    for batch in train_set:
        lab_batch = rgb2lab(batch[0])
        X_batch = lab_batch[:,:,:,0] 
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [ ]:
#MODEL
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))

model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))

model.add(Conv2D(512, (3,3), activation='relu', padding='same'))

model.add(Conv2D(512, (3,3), activation='relu', padding='same'))

model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))

model.add(UpSampling2D((2, 2)))
# model.add(Dropout(0.2))


model.add(Conv2D(64, (3,3), activation='relu', padding='same'))

model.add(UpSampling2D((2, 2)))
# model.add(Dropout(0.2))


model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
# model.add(Dropout(0.2))

model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
# model.add(Dropout(0.2))

model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

# model.add(UpSampling2D((2, 2)))



In [ ]:
opt=keras.optimizers.Adam(learning_rate= 8e-5)


model.compile(optimizer=opt, loss='mse' , metrics=['accuracy'])
model.summary()

In [ ]:
filepath = "soutenance.model"
checkpoint = ModelCheckpoint(filepath,
                             save_best_only=True,
                             monitor='loss',
                             mode='min')

In [ ]:
valeurs=model.fit(x=gen_ab_images(train_set), 
          callbacks=[checkpoint], 
          epochs=N_EPOCHS, 
          validation_data=gen_ab_images(valid_set), 
          steps_per_epoch=len(train_set),
          validation_steps=len(valid_set),
          shuffle=True)

In [ ]:
model.save("soutenance.model")

# Test

In [ ]:
# display the history values 

plt.plot(valeurs.history['accuracy'])
plt.plot(valeurs.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# afficher l'historique des valeurs de Loss

plt.plot(valeurs.history['loss'])
plt.plot(valeurs.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:


model=keras.models.load_model("soutenance.model",
                        custom_objects=None,
                        compile=True)

In [ ]:
test_data_gen = ImageDataGenerator(rescale = 1./255)

test_set = test_data_gen.flow_from_directory('/content/drive/MyDrive/testing',
                                              target_size = (IMG_SIZE, IMG_SIZE),
                                              batch_size = 1000)

In [ ]:

def transformer_en_lab(data):
  X=[]
  Y=[]

  for img in tqdm(data[0]):
    try:
      lab=rgb2lab(img)
      X.append(lab[:,:,0])
      
      Y.append(lab[:,:,1:]/128)

    except:
      pass
  X=np.array(X)  
  X=X.reshape(X.shape+(1,))  
  Y=np.array(Y)

  return X,Y
  
X_test=[]
Y_test=[]


X_test,Y_test=transformer_en_lab(test_set[0])


In [ ]:
X_test.shape

In [ ]:

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test,Y_test)
print("test loss, test acc:", results)



In [ ]:
output = model.predict(X_test)
output = output * 128

In [ ]:

random_indices = [random.randint(0, 13) for i in range(9)]

plt.figure(figsize=(10,10))

for i, index in enumerate(random_indices):  
    image = np.zeros((256, 256, 3))
    image[:,:,0] = X_test[index][:,:,0]
    image[:,:,1:] = output[index]
    rgb=lab2rgb(image)

    plt.subplot(3,3,i+1)
    plt.imshow(rgb)
    # plt.title(f"Predicted: {pred}, \n Class: {actual}")
    plt.tight_layout()